# Multilayer Perceptron
## (Aritifical Neural Network / Neural Network)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from sklearn.metrics import mean_squared_error

## 1) Load dataset and split the data into training and testing

In [ ]:
df = pd.read_csv("datasets/wine.csv")
# df = df.drop("Wine", axis=1)
df

In [ ]:
np.random.seed(3)

y_fitting = df["Proline"].to_numpy(dtype=float)
X_fitting = df[["Alcohol", "Phenols", "Flavanoids", "Nonflavanoid.phenols"]].to_numpy(dtype=float)      

print(np.shape(X_fitting), np.shape(y_fitting))

In [ ]:
# Because the some values are quite weird, we have to mmodify them
# X_fitting[:, 1] *= 10.0     # Volatile Acidity
# X_fitting[:, 2] *= 10.0     # Citric Acid
# X_fitting[:, 4] *= 100.0    # Chlorine
# X_fitting[:, 5] /= 5        # Free Sulfur Dioxide
# X_fitting[:, 6] /= 5        # Total Sulfur Dioxide

X_fitting[:, 0] -= 13.0     # Alcohol
y_fitting[:] /= 100

In [ ]:
# plt.scatter(X_fitting[:, 5] * 5, y_fitting, color = "blue")
# plt.xlabel("Free Sulfur Dioxide")
# plt.ylabel("Quality")
# plt.show()

# plt.scatter(X_fitting[:, 3], y_fitting, color = "blue")
# plt.xlabel("Residual Sugar")
# plt.ylabel("Quality")
# plt.show()

plt.scatter(X_fitting[:, 0] * 5, y_fitting, color = "blue")
plt.xlabel("Alcohol")
plt.ylabel("Proline")
plt.show()

plt.scatter(X_fitting[:, 1], y_fitting, color = "blue")
plt.xlabel("Phenols")
plt.ylabel("Proline")
plt.show()

#### 2) Start training

In [ ]:
def ReLU(x):
    size = np.shape(x)
    for i in range(size[0]):
        for j in range(size[1]):
            if x[i][j] < 0:
                x[i][j] = 0
    return x


def LeakyReLU(x):
    size = np.shape(x)
    for i in range(size[0]):
        for j in range(size[1]):
            if x[i][j] < 0:
                x[i][j] *= 0.1
    return x


# def softMax(x):


In [ ]:
def lossFunc(error):
    sum = 0.0
    for i in error:
        sum += i * i
    return sum / 2 / np.size(error[0])

In [ ]:
# Gradient descent
# dLoss/dW[2] = dLoss/dY[3] * dY[3]/dZ[2] * dZ[2]/dW[2]
#             = Error       * 1           * Y[2]
# dLoss/dB[2] = dLoss/dY[3] * dY[3]/dZ[2] * dZ[2]/dB[2]
#             = Error       * 1

# dLoss/dW[1] = dLoss/dY[3] * dY[3]/dZ[2] * dZ[2]/dY[2] * dY[2]/dZ[1] * dZ[1]/dW[1]
#             = Error       * 1           * W[2]        * 1           * Y[1]
# dLoss/dB[1] = dLoss/dY[3] * dY[3]/dZ[2] * dZ[2]/dY[2] * dY[2]/dZ[1] * dZ[1]/dB[1]
#             = Error       * 1           * W2          * 1

# dLoss/dW[0] = dLoss/dY[3] * dY[3]/dZ[2] * dZ[2]/dY[2] * dY[2]/dZ[1] * dZ[1]/dY[1] * dY[1]/dZ[0] * dZ[0]/dW[0]
#             = Error       * 1           * W[2]        * 1           * W[1]        * 1           * Y[0]
# dLoss/dB[0] = dLoss/dY[3] * dY[3]/dZ[2] * dZ[2]/dY[2] * dY[2]/dZ[1] * dZ[1]/dB[1] * dY[1]/dZ[0] * dZ[0]/dB[0]
#             = Error       * 1           * W2         * 1            * W[1]        * 1

In [ ]:
def multilayerPerceptron(X_calc, y_calc, activeFunc = ReLU, hddn = [5], learning_rate = 0.05):

    # Toggle size
    size_X = np.shape(X_calc)
    size_y = np.shape(y_calc)
    if len(size_y) == 1:    # if y contains only one target
        size_y = (size_y[0], 1)
        y_calc = np.reshape(y_calc, (size_y[0], 1))

    # Make all matrix to support the ANN
    W = [None for _ in range(len(hddn) + 1)]
    B = [None for _ in range(len(hddn) + 1)]
    Y = [None for _ in range(len(hddn) + 2)]     # Two

    s_iter = [ *[ size_X[1] ], *[ hddn[i] for i in range(len(hddn)) ], *[ size_y[1] ] ]
    for i in range(len(W)):
        W[i] = np.random.normal(0, 1, size = (s_iter[i], s_iter[i + 1]))
        B[i] = np.random.normal(0, 1, size = (1, s_iter[i + 1]))



    # For error
    y_predicted = np.zeros(size_y[0])
    error_points = np.zeros(100)

    # Start running
    step = 0
    while step < 100:

        mix = np.random.permutation(size_X[0])
        for i in mix:

            # Y[0] = X
            # Z[0] = Y[0] * W[0] + B[0]
            # Y[1] = activeFunc(Z[0])

            # Z[1] = Y[1] * W[1] + B[1]
            # Y[2] = activeFunc(Z[1])

            # Z[2] = Y[2] * W[2] + B[2]
            # Y[3] = activeFunc(Z[2])

            Y[0] = np.reshape(X_calc[i], (1, size_X[1]))
            for j in range(len(hddn) + 1):
                z        = np.dot(Y[j], W[j]) + B[j]
                Y[j + 1] = np.array(activeFunc(z))

            y_predicted[i] = Y[-1]

            # dLoss/dW[2] = Y[2] * Error
            # dLoss/dW[1] = Y[1] * Error * W[2]
            # dLoss/dW[0] = Y[0] * Error * W[2] * W[1]

            error = 2 / size_X[0] * (y_predicted[i] - y_calc[i])
            for j in range(len(W) - 1, -1, -1):
                multi = np.reshape(error, (1, 1))   if (j == len(W) - 1)   else np.dot(multi, W[j + 1].T)
                W[j] = W[j] - learning_rate * np.dot(Y[j].T, multi)
                B[j] = B[j] - learning_rate * multi
            
            #print(error)

        try:
            rmse = mean_squared_error(y_calc, y_predicted, squared = False)
            error_points[step] = rmse
            step += 1
            print("Step ", step, " done with RMSE = ", rmse)
        except:
            for i in range(len(W)):
                W[i] = np.random.normal(0, 1, size = (s_iter[i], s_iter[i + 1]))
                B[i] = np.random.normal(0, 1, size = (1, s_iter[i + 1]))
            print("RESET at step ", step)
            step = 0
    
    plt.plot(np.arange(100), error_points)
    plt.show()
    
    return W, B



In [ ]:
hddn = [12, 10]
active_func = ReLU
W, B = multilayerPerceptron(X_fitting, y_fitting, hddn = hddn, activeFunc = active_func, learning_rate = 0.0005)
#W1, W2, B1, B2 = trainDataMLP(X_train, y_train, learning_rate = 0.5)
W, B

In [ ]:
# Make predictions
def getPredictedY(X_calc, W, B, hddn, activeFunc = ReLU):

    size_X = np.shape(X_calc)
    y_predicted = [None for _ in range(size_X[0])]
    
    for i in range(size_X[0]):

        Y = [None for _ in range(len(hddn) + 2)]
        Y[0] = np.reshape(X_calc[i], (1, np.shape(X_calc)[1]))

        for j in range(len(hddn) + 1):
            z        = np.dot(Y[j], W[j]) + B[j]
            Y[j + 1] = np.array(activeFunc(z))
        
        y_predicted[i] = Y[-1]
    
    return y_predicted

pred_y = np.array(getPredictedY(X_fitting, W, B, hddn = hddn, activeFunc = active_func))
# pred_y

In [ ]:
plt.scatter(X_fitting[:, 0], y_fitting, color = "blue")
plt.scatter(X_fitting[:, 0], pred_y, color = "red")
plt.xlabel("Alcohol")
plt.ylabel("Proline")
plt.show()

plt.scatter(X_fitting[:, 1], y_fitting, color = "blue")
plt.scatter(X_fitting[:, 1], pred_y, color = "red")
plt.xlabel("Phenols")
plt.ylabel("Proline")
plt.show()

In [ ]:
# Predict all of values
true_pos = 0
true_neg = 0
fals_pos = 0
fals_neg = 0

for i in range(instan_train):
    pred_y = getPredictedY(X_train[i], W1, W2, B1, B2)
    if pred_y >= 0.5:   # Prediction is True
        if y_train[i] == 1.0:
            true_pos += 1
        else:
            fals_pos += 1
    else:
        if y_train[i] == 1.0:
            fals_neg += 1
        else:
            true_neg += 1


print(true_pos, fals_neg)
print(fals_pos, true_neg)